In [69]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: AutoML tabular forecasting model for batch prediction

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use the Vertex SDK to create tabular forecasting models and do batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

### Dataset

The dataset used for this tutorial a time series dataset containing samples drawn from the Iowa Liquor Retail Sales dataset. Data were made available by the Iowa Department of Commerce. It is provided under the Creative Commons Zero v1.0 Universal license. For more details, see: https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales. This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in BigQuery.

### Objective

In this tutorial, you create an AutoML tabular forecasting model from a Python script, and then do a batch prediction using the Vertex SDK. You can alternatively create and deploy models using the `gcloud` command-line tool or online using the Cloud Console.

The steps performed include:

- Create a Vertex `Dataset` resource.
- Train the model.
- View the model evaluation.
- Make a batch prediction.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Google Cloud Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex SDK for Python.

In [70]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG
! pip3 install --upgrade tensorflow $USER_FLAG

E0413 18:45:42.224843356   13171 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


E0413 18:45:45.098474626   13171 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [71]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [9]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [6]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [7]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://vertex-ai-devaip-20220413184619/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [10]:
import google.cloud.aiplatform as aiplatform

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [11]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Tutorial

Now you are ready to start creating your own AutoML tabular forecasting model.

#### Location of BigQuery training data.

Now set the variable `TRAINING_DATASET_BQ_PATH` to the location of the BigQuery table. 

In [12]:
TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `TimeSeriesDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the `Dataset` resource.

This operation may take several minutes.

In [13]:
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train" + "_" + TIMESTAMP,
    bq_source=[TRAINING_DATASET_BQ_PATH],
)

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

print(dataset.resource_name)

Creating TimeSeriesDataset
Create TimeSeriesDataset backing LRO: projects/931647533046/locations/us-central1/datasets/1781503506113363968/operations/4217474334949638144
TimeSeriesDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/1781503506113363968
To use this TimeSeriesDataset in another session:
ds = aiplatform.TimeSeriesDataset('projects/931647533046/locations/us-central1/datasets/1781503506113363968')
projects/931647533046/locations/us-central1/datasets/1781503506113363968


In [14]:
COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

### Create and run training job

To train an AutoML model, you perform two steps: 1) create a training job, and 2) run the job.

#### Create training job

An AutoML training job is created with the `AutoMLForecastingTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the job for the training pipeline.

In [18]:
MODEL_DISPLAY_NAME = "iowa-liquor-sales-forecast-model_{TIMESTAMP}"

training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

#### Run the training pipeline

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `time_column`:
- `time_series_identifier_column`:

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take up to one hour.

In [19]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name=MODEL_DISPLAY_NAME,
    predefined_split_column_name=None,
)

No dataset split provided. The service will use a default split.
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5211248059786199040?project=931647533046
AutoMLForecastingTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLForecastingTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040 current state:
PipelineState.PIPELINE_ST

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [20]:
# Get model resource ID
models = aiplatform.Model.list(filter=f"display_name={MODEL_DISPLAY_NAME}")
model = models[0]

# Get a reference to the Model Service client
client_options = aiplatform.initializer.global_config.get_client_options()
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=model.resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

name: "projects/931647533046/locations/us-central1/models/711942575077982208/evaluations/3235464588073050687"
metrics_schema_uri: "gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml"
metrics {
  struct_value {
    fields {
      key: "meanAbsoluteError"
      value {
        number_value: 8041.1226
      }
    }
    fields {
      key: "meanAbsolutePercentageError"
      value {
        number_value: 98.682396
      }
    }
    fields {
      key: "rSquared"
      value {
        number_value: 0.064227924
      }
    }
    fields {
      key: "rootMeanSquaredError"
      value {
        number_value: 14908.694
      }
    }
    fields {
      key: "rootMeanSquaredLogError"
      value {
        number_value: 5.49582
      }
    }
  }
}
create_time {
  seconds: 1649878893
  nanos: 58602000
}



## Send a batch prediction request

Send a batch prediction to your deployed model.

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `instances_format`: The format for the input instances, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `predictions_format`: The format for the output predictions, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [21]:
import os

from google.cloud import bigquery

batch_predict_bq_output_dataset_name = f"iowa_liquor_sales_predictions_{TIMESTAMP}"
batch_predict_bq_output_dataset_path = "{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client(project=PROJECT_ID)
bq_dataset = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_region = "US"  # @param {type : "string"}
bq_dataset.location = dataset_region
bq_dataset = client.create_dataset(bq_dataset)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_region
    )
)

Created bigquery dataset vertex-ai-dev.iowa_liquor_sales_predictions_20220413184619 in US


In [38]:
PREDICTION_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2021_sales_predict"
)

batch_prediction_job = model.batch_predict(
    job_display_name=f"iowa_liquor_sales_forecasting_predictions_{TIMESTAMP}",
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    #bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    gcs_destination_prefix=BUCKET_URI,
    predictions_format="bigquery",
    sync=False,
)

print(batch_prediction_job)

Creating BatchPredictionJob
<google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f5399d754d0> is waiting for upstream dependencies to complete.
BatchPredictionJob created. Resource name: projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5131731378865438720?project=931647533046
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_PENDING


### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, you can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [39]:
batch_prediction_job.wait()

BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/5131731378865438720 current state:
JobState.JOB_STATE_SUCCEEDED
BatchPredictionJob run completed. Resource name: projects/931647533046/locations/us-central1/batchPredictionJobs/513173137886543

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a CSV format:

- CSV header + predicted_label
- CSV row + prediction, per prediction request

In [40]:
import tensorflow as tf

bp_iter_outputs = batch_prediction_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

city,county,date,sale_dollars,store_name,zip_code,predicted_sale_dollars

Burlington,DES MOINES,2021-04-16,null,River Mart,52601,24.338449478149414

Manchester,DELAWARE,2021-04-28,null,Casey's General Store #2624 / Manchester,52057,22.749940872192383

Webster City,HAMILTON,2021-04-06,null,Fareway Stores #395 / Webster City,50595,25.622598648071289

Muscatine,MUSCATINE,2021-04-26,null,CIRCLE K #4706600 / MUSCATINE,52761,20.759757995605469

Urbandale,POLK,2021-04-01,null,Hy-Vee Food Store / Urbandale,50322,33.2498779296875

Centerville,APPANOOSE,2021-04-30,null,Wal-Mart 1621 / Centerville,52544,26.7572021484375

Charles City,FLOYD,2021-04-20,null,Rush Stop / Charles City,50616,22.415464401245117

Forest City,WINNEBAGO,2021-04-15,null,Hy-Vee - Forest City,50436,29.969043731689453

Doon,LYON,2021-04-21,null,R&L Foods,51235,23.884374618530273

Huxley,STORY,2021-04-29,null,Fareway Stores #155 / Huxley,50124,27.750967025756836

Winterset,MADISON,2021-04-20,null,Rodgers Spirits and More,50273,

### Visualize the forecasts

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).
The code block included in this section dynamically generates a Data Studio link that specifies the template, the location of the forecasts, and the query to generate the chart. The data is populated from the forecasts generated earlier.

You can inspect the used template at https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8. This was created by Google specifically to view forecasting predictions.

In [41]:
import urllib

tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        PREDICTION_DATASET_BQ_PATH,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

https://datastudio.google.com/c/u/0/reporting?params=%7B%22templateId%22%3A%22067f70d2-8cd6-4a4c-a099-292acd1053e8%22%2C%22ds0.connector%22%3A%22BIG_QUERY%22%2C%22ds0.projectId%22%3A%22vertex-ai-dev%22%2C%22ds0.billingProjectId%22%3A%22vertex-ai-dev%22%2C%22ds0.type%22%3A%22CUSTOM_QUERY%22%2C%22ds0.sql%22%3A%22SELECT+%5Cn+CAST%28input.date+as+DATETIME%29+timestamp_col%2C%5Cn+CAST%28input.store_name+as+STRING%29+time_series_identifier_col%2C%5Cn+CAST%28input.sale_dollars+as+NUMERIC%29+historical_values%2C%5Cn+CAST%28predicted_sale_dollars.value+as+NUMERIC%29+predicted_values%2C%5Cn+%2A+%5CnFROM+%60bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict%60+input%5CnLEFT+JOIN+%60vertex-ai-dev.iowa_liquor_sales_predictions_20220413184619.predictions_2022_04_13T12_41_40_622Z%60+output%5CnON%5CnCAST%28input.date+as+DATETIME%29+%3D+CAST%28output.date+as+DATETIME%29%5CnAND+CAST%28input.store_name+as+STRING%29+%3D+CAST%28output.store_name+as+STRING%29%22%7D


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- AutoML Training Job
- Model
- Batch Prediction Job
- Cloud Storage Bucket

In [42]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete dataset
dataset.delete()

# Training job
training_job.delete()

# Delete model
model.delete()

# Delete batch prediction job
batch_prediction_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

Deleting TimeSeriesDataset : projects/931647533046/locations/us-central1/datasets/1781503506113363968
Delete TimeSeriesDataset  backing LRO: projects/931647533046/locations/us-central1/operations/2550157310404067328
TimeSeriesDataset deleted. . Resource name: projects/931647533046/locations/us-central1/datasets/1781503506113363968
Deleting AutoMLForecastingTrainingJob : projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040
Delete AutoMLForecastingTrainingJob  backing LRO: projects/931647533046/locations/us-central1/operations/7693268084861173760
AutoMLForecastingTrainingJob deleted. . Resource name: projects/931647533046/locations/us-central1/trainingPipelines/5211248059786199040
Deleting Model : projects/931647533046/locations/us-central1/models/711942575077982208
Delete Model  backing LRO: projects/931647533046/locations/us-central1/operations/6936663347462930432
Model deleted. . Resource name: projects/931647533046/locations/us-central1/models/71194257507